# CIFAR 10

This notebook trains a classifier on CIFAR10 using Tensorflow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import os.path
import time
import datetime

In [ ]:
# 100 epochs will take ages on CPU, reduce or use GPU
EPOCHS = 50
BATCH_SIZE = 1000
LABELS = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

## Load Data

In [ ]:
def prepare(batch):
    return (
        {
            "image": tf.cast(batch["image"], tf.float32) / 255.,
            "id": batch["id"],
        },
        batch["label"],
    )

In [ ]:
train_set, test_set = tfds.load("cifar10", split=["train", "test"], data_dir="/tmp/tensorflow_datasets")

In [ ]:
# split off a chunk of training data for validation
val_ds = train_set.take(1000)
train_set = train_set.skip(1000)

train_ds = (
    train_set
    .shuffle(buffer_size=8 * BATCH_SIZE)
    .batch(BATCH_SIZE)
    .map(prepare)
)
val_ds = (
    val_ds
    .batch(BATCH_SIZE)
    .map(prepare)
)
test_ds = (
    test_set
    .batch(BATCH_SIZE)
    .map(prepare)
)

In [ ]:
# visualise examples
batch, labels = next(iter(train_ds))
images = batch["image"].numpy()

fig, axes = plt.subplots(2, ncols=5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='binary', interpolation='nearest')
    ax.set_title(LABELS[labels[i]])

## Train Model

### Defining all test models

In [ ]:
def build_model(name="model"):
    block = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same", input_shape=(32, 32, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Reshape((2048,)),
        tf.keras.layers.Dense(10)
    ], name=name)
    
    input_ = tf.keras.Input(name="image", shape=(32, 32, 3))
    output = block(input_)
    model = tf.keras.Model(inputs=input_, outputs=output)
    
    model.compile(
        "adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["sparse_categorical_accuracy"])
    
    return model

Creating a checkpoint path to store model trainig history

In [ ]:
# checkpoint callback
checkpoint_path = "cp_training_cifar_10/cp-{epoch:04d}"
checkpoint_dir = os.path.dirname(checkpoint_path)

BATCHES_PER_EPOCH = int(49000/BATCH_SIZE)

# creating a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_freq=1*BATCHES_PER_EPOCH)

Instantiating a CNN model on the cifar-10 dataset 

In [ ]:
model = build_model()
model.summary()

# save initialised model
model.save(checkpoint_path.format(epoch=0))

### Training model

In [ ]:
start = time.time()
history = model.fit(train_ds, 
                    epochs=EPOCHS, 
                    validation_data=val_ds,
                    verbose=2,
                    shuffle=False,
                    callbacks=[cp_callback, tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=20)])
end = time.time()
print("Total time:", datetime.timedelta(seconds=end-start))

## Results
Summary of the model performance. The defined methods can be used to compare multiple models.

In [ ]:
# model plotter
def plot_model_results(history, clr, i="_alt"):
    ax[0].plot(history.history["loss"], "{}".format(clr), label="M{} Train loss".format(i), linewidth=2)
    ax[0].plot(history.history["val_loss"], "{}--".format(clr), label="M{} Val loss".format(i), linewidth=2)
    ax[1].plot(history.history["sparse_categorical_accuracy"], "{}".format(clr), label="M{} Train accuracy".format(i), linewidth=1.5)
    ax[1].plot(history.history["val_sparse_categorical_accuracy"], "{}--".format(clr), label="M{} Val accuracy".format(i), linewidth=1.5)
    ax[0].set_xlabel("$Epochs$", fontsize=16), ax[1].set_xlabel("$Epochs$", fontsize=16)
    ax[0].set_ylabel("$Loss$", fontsize=16), ax[1].set_ylabel("$Accuracy$", fontsize=16)
    ax[0].set_title("Loss", fontsize=18), ax[1].set_title("Accuracy", fontsize=18)
    ax[0].legend(frameon=False, fontsize=14), ax[1].legend(frameon=False, fontsize=14)
    
# print results
def return_results(model, test_ds, i="0"):
    # Evaluate model comparison
    test_batch, test_labels = next(iter(test_ds))
    loss, acc = model.evaluate(test_batch, test_labels, verbose=0)
    print("M{}|| Accuracy: {:.2f}% --- Loss: {:.2f}".format(i, 100 * acc, loss))

In [ ]:
# print result summary 
return_results(model, test_ds)

# create a loss & accuracy subplot
f, ax = plt.subplots(figsize=(14, 6), ncols=2)

# plot results of each model
plot_model_results(history, "g", 1)

## Self Influence 
#### Incorprating the self-influence code outlined in [TracIn paper](https://github.com/frederick0329/TracIn/blob/master/imagenet/resnet50_imagenet_self_influence.ipynb). 

Method below calculations the self-influence (memorisation) score of a given training example for a given batch of images & labels and model checkpoints. Outputs from **run_self_influence** are concatenated into a dictionary using **memorisation_results** to simplify output. The indexing can be taken as the ID across each result member.

In [ ]:
# method to calculate self-influence of batch members.
@tf.function
def run_self_influence(images, labels, models):
    self_influences = []
    for m in models:
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(m.trainable_weights)
            probs = m(images, training=False)
            loss = tf.keras.losses.sparse_categorical_crossentropy(labels, probs, from_logits=True)
        grads = tape.jacobian(loss, m.trainable_weights)
        scores = tf.add_n([tf.math.reduce_sum(
            grad * grad, axis=tf.range(1, tf.rank(grad), 1)) 
            for grad in grads])
        self_influences.append(scores)  

    # using probs from last checkpoint
    probs, predicted_labels = tf.math.top_k(probs, k=1)
    return tf.math.reduce_mean(tf.stack(self_influences, axis=-1), axis=-1), labels, probs, predicted_labels

# method to concatenate all of the batch results together
def memorisation_results(memorisation, images, probs, labels, predicted_labels):
    result_dictionary = {
        "memorisation": np.array(np.concatenate(memorisation)),
        "images": np.concatenate(images),
        "probs": np.concatenate(probs),
        "labels": np.concatenate(labels),
        "predicted_labels": np.concatenate(predicted_labels)
    }
    return result_dictionary

This method incorprates **run_self_influence** and **memorisation_results** to return results for any given model scenario. It is used to study the comparisons between different CP memorisation scores.

In [ ]:
def batch_self_influence(train_ds, models):
    memorisation_list = []
    image_list = []
    probs_list = []
    labels_list = [] 
    predicted_labels_list = []

    start = time.time()
    for batch, labels in train_ds:
        memorisation_score, labels, probs, predictied_labels = run_self_influence(batch["image"], labels, models)
        memorisation_list.append(memorisation_score)
        image_list.append(batch["image"])
        probs_list.append(probs)
        labels_list.append(labels)
        predicted_labels_list.append(predictied_labels)
    end = time.time()
    print("Total time:", datetime.timedelta(seconds=end - start))
    
    return memorisation_results(memorisation_list, image_list, probs_list, labels_list, predicted_labels_list)

A method to load the desired model weights of a single or a list of epochs.

In [ ]:
def return_models(epochs):
    loaded_models = []
    for epoch in epochs:
        path = "{}/cp_training_cifar_10/cp-00{:02d}".format(os.getcwd(), epoch)
        if os.path.exists(path) == False:
            print ("File not found: cp-00{:02d}".format(epoch))
        else:
            model = tf.keras.models.load_model("cp_training_cifar_10/cp-00{:02d}".format(epoch))
            loaded_models.append(model)
    return loaded_models

In [ ]:
def get_results(train_ds, models):
    results = [] 
    if len(models) > 1:
        for model in models:
            results.append(batch_self_influence(train_ds, [model]))
    results.append(batch_self_influence(train_ds, models[1:])) # ave. self-influence
    return results

### Saving results

Load models from different epochs to calculate CP memorisation

In [ ]:
# redefine training set with a reduced batch size for memory issues
train_influence = (
    train_set
    .batch(50)
    .map(prepare)
)

In [ ]:
# model index is later used to label plots
model_cps = [0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
models = return_models(model_cps)

In [ ]:
# returns all model results in a list 
#  > zeroth index holds averaged CP
results = get_results(train_influence, models)

In [ ]:
import pickle

# save options
EXTENSION = "scan_results_shuffled_1000"

# store data (serialize)
with open('results/{}.pickle'.format(EXTENSION), 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
